In [7]:
import ctypes
import logging
from PIL import Image, ImageDraw
#from observation_panel import get_observation_panel
from info_panel import get_info_panel
from forecast_panel import get_forecasts_panel
from celestial_panel import get_celestial_panel
from sensor_panel import get_sensor_panel
#from sensor_data import get_sensor_data
from timeit import default_timer as timer
from configparser import SectionProxy
from typing import Optional
from type_alias import Icons, Fonts
from multiprocessing import Process
from weather import get_observation_data, get_forecast_data, get_radiation_data
import log
import logging
import utils
from icons import get_weather_images
import configparser
import refresh



In [8]:
CONFIG_FILENAME = 'config.ini'
log.setup()
logger = logging.getLogger(__name__)
logger.info("App starting")
utils.check_python_version()
logger.info(f'Reading config file "{CONFIG_FILENAME}"')
with open(CONFIG_FILENAME) as f:
    config_parser = configparser.ConfigParser()
    config_parser.read_file(f)
    logger.info('Config: %s', config_parser.items('general'))
    config = config_parser['general']

    fonts = utils.get_fonts(config)
    images = get_weather_images()

    logger.info('Import epd control library')
    (epd_so, panel_size) = utils.get_epd_data(config)

    logger.info("Initial refresh")
    refresh.refresh(panel_size, fonts, images, config, epd_so, True)  # Once in the beginning


2022-08-12 23:49:45 - __main__ - INFO - App starting
2022-08-12 23:49:45 - __main__ - INFO - Reading config file "config.ini"
